<a href="https://colab.research.google.com/github/alina518/weather-app/blob/main/weather_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
!pip install streamlit pyngrok pydantic pydantic-ai requests nest_asyncio --quiet


In [2]:
# 2️⃣ Set up ngrok (you need a verified ngrok account)
# Replace YOUR_AUTHTOKEN_HERE with your ngrok authtoken
!ngrok authtoken '38IKzIWCtrSFP1YoCbohpVWdfsA_4cSm6ktEcCUufMAf2pkYm'


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:

%%writefile weather_app.py
import streamlit as st
import requests
from datetime import datetime
from pydantic import BaseModel

# ================== CONFIG ==================
OPENWEATHER_API_KEY = "96fc59ab86b1aaa414f33f209692d58f"  # <-- put your key here

st.set_page_config(page_title="🌤️ Weather Forecast", layout="centered")

# ================== GLOBAL STYLES ==================
st.markdown("""
<style>
.stApp {
    background-image: url("https://images.unsplash.com/photo-1506744038136-46273834b3fb?auto=format&fit=crop&w=1600&q=80");
    background-size: cover;
    background-position: center;
    background-attachment: fixed;
    background-color: rgba(0,0,0,0.4);
    background-blend-mode: darken;
}

.weather-card {
    background: rgba(255, 255, 255, 0.95);
    border-radius: 20px;
    padding: 20px;
    text-align: center;
    box-shadow: 0 8px 20px rgba(0,0,0,0.15);
    margin: 10px;
    transition: transform 0.2s ease;
    position: relative;
}
.weather-card:hover {
    transform: translateY(-5px);
}

.weather-header {
    font-size: 18px;
    font-weight: 600;
    margin-bottom: 6px;
}
.emoji {
    font-size: 28px;
    vertical-align: middle;
    margin-right: 6px;
}
.date {
    vertical-align: middle;
}
.description {
    font-size: 14px;
    opacity: 0.85;
    margin-bottom: 10px;
}
.temp {
    font-size: 34px;
    font-weight: 700;
    margin: 6px 0 4px;
}
.feels {
    font-size: 13px;
    opacity: 0.8;
    margin-bottom: 8px;
}
.divider {
    border: none;
    height: 1px;
    background: rgba(0,0,0,0.1);
    margin: 8px 0;
}
.details {
    font-size: 12px;
    line-height: 1.6;
    opacity: 0.85;
}

.tooltip {
    position: relative;
    display: inline-block;
    cursor: pointer;
}

.tooltip .tooltiptext {
    visibility: hidden;
    width: 90px;
    background-color: rgba(0,0,0,0.8);
    color: #fff;
    text-align: center;
    border-radius: 6px;
    padding: 4px 6px;
    position: absolute;
    z-index: 1;
    bottom: 125%; /* position above icon */
    left: 50%;
    transform: translateX(-50%);
    opacity: 0;
    transition: opacity 0.2s;
    font-size: 11px;
}

.tooltip:hover .tooltiptext {
    visibility: visible;
    opacity: 1;
}

/* Animations */
@keyframes pulse {0% {transform: scale(1);} 50% {transform: scale(1.2);} 100% {transform: scale(1);}}
@keyframes swing {0% {transform: rotate(0deg);} 25% {transform: rotate(10deg);} 50% {transform: rotate(0deg);} 75% {transform: rotate(-10deg);} 100% {transform: rotate(0deg);}}
@keyframes float {0% {transform: translateY(0px);} 50% {transform: translateY(-5px);} 100% {transform: translateY(0px);}}

.emoji.sun { animation: swing 3s infinite ease-in-out; }
.emoji.cloud { animation: float 4s infinite ease-in-out; }
.emoji.rain { animation: float 3s infinite ease-in-out; }
.emoji.snow { animation: float 4s infinite ease-in-out; }
.emoji.storm { animation: swing 2s infinite ease-in-out; }
.details .tooltip span { animation: pulse 2s infinite ease-in-out; }

</style>
""", unsafe_allow_html=True)

# ================== DATA MODEL ==================
class DailyForecast(BaseModel):
    date: str
    description: str
    temp: float
    feels_like: float
    humidity: int
    wind_speed: float
    sunrise: str
    sunset: str
    emoji: str
    color: str
    icon_class: str

# ================== WEATHER FETCH ==================
def get_3day_forecast(city: str):
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city, "appid": '96fc59ab86b1aaa414f33f209692d58f', "units": "metric"}
    res = requests.get(url, params=params).json()
    if res.get("cod") != "200":
        raise ValueError(res.get("message", "Failed to fetch weather"))

    forecasts, used_dates = [], set()
    for item in res["list"]:
        dt = datetime.fromtimestamp(item["dt"])
        if dt.hour != 12: continue
        date_str = dt.strftime("%a, %b %d")
        if date_str in used_dates: continue
        used_dates.add(date_str)

        desc = item["weather"][0]["description"].capitalize()
        temp = item["main"]["temp"]
        feels = item["main"]["feels_like"]
        humidity = item["main"]["humidity"]
        wind_speed = item["wind"]["speed"]

        # Emoji and icon class
        d = desc.lower()
        if "clear" in d: emoji, icon_class = "🌞", "sun"
        elif "cloud" in d: emoji, icon_class = "⛅", "cloud"
        elif "rain" in d: emoji, icon_class = "🌧️", "rain"
        elif "snow" in d: emoji, icon_class = "❄️", "snow"
        elif "storm" in d or "thunder" in d: emoji, icon_class = "⛈️", "storm"
        else: emoji, icon_class = "🌤️", "sun"

        # Temperature color
        if temp <= 0: color = "#00BFFF"
        elif temp <= 15: color = "#1E90FF"
        elif temp <= 25: color = "#32CD32"
        elif temp <= 35: color = "#FFA500"
        else: color = "#FF4500"

        forecasts.append(DailyForecast(
            date=date_str, description=desc, temp=temp, feels_like=feels,
            humidity=humidity, wind_speed=wind_speed,
            sunrise=datetime.fromtimestamp(res["city"]["sunrise"]).strftime("%H:%M"),
            sunset=datetime.fromtimestamp(res["city"]["sunset"]).strftime("%H:%M"),
            emoji=emoji, color=color, icon_class=icon_class
        ))
        if len(forecasts) == 3: break
    return forecasts

# ================== UI ==================
st.title("🌤️ Weather Forecast")

city = st.text_input("Enter city name")

if city:
    try:
        st.info(f"Fetching weather for **{city}**...")
        forecast_list = get_3day_forecast(city)
        cols = st.columns(3)
        for i, fc in enumerate(forecast_list):
            with cols[i]:
                st.markdown(f"""
                <div class="weather-card">
                    <div class="weather-header">
                        <span class="emoji {fc.icon_class}">{fc.emoji}</span>
                        <span class="date">{fc.date}</span>
                    </div>
                    <div class="description">{fc.description}</div>
                    <div class="temp" style="color:{fc.color};">{fc.temp:.1f}°C</div>
                    <div class="feels">Feels like {fc.feels_like:.1f}°C</div>
                    <hr class="divider">
                    <div class="details">
                        <span class="tooltip">💧 {fc.humidity}%
                            <span class="tooltiptext">Humidity</span>
                        </span> &nbsp;&nbsp;
                        <span class="tooltip">🌬️ {fc.wind_speed} m/s
                            <span class="tooltiptext">Wind speed</span>
                        </span><br>
                        <span class="tooltip">🌅 {fc.sunrise}
                            <span class="tooltiptext">Sunrise</span>
                        </span> &nbsp;&nbsp;
                        <span class="tooltip">🌇 {fc.sunset}
                            <span class="tooltiptext">Sunset</span>
                        </span>
                    </div>
                </div>
                """, unsafe_allow_html=True)

    except Exception as e:
        st.error(f"❌ {e}")



Overwriting weather_app.py


In [4]:
from pyngrok import ngrok
import time
import os

# Kill any old tunnels
ngrok.kill()

# Run Streamlit in background
os.system("nohup streamlit run weather_app.py --server.port 8501 --server.address 0.0.0.0 --server.headless true &")

# Wait a few seconds to let Streamlit start
time.sleep(5)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print("🌐 Streamlit URL:", public_url)

🌐 Streamlit URL: NgrokTunnel: "https://transriverina-unrenovated-patti.ngrok-free.dev" -> "http://localhost:8501"
